In [14]:
# Experimenting environment
from bs4 import BeautifulSoup

html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

i = '<p>hallo welt</p>'
s = BeautifulSoup(html_doc, 'html.parser')
print(s.title.namee)



o = i.replace('welt', '<b>welt</b>')
#print(o)



None


In [93]:
from bs4 import BeautifulSoup

i = """<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type='text/xsl' href='../../Ressourcen/Stylesheet.xsl'?>
<TEI xmlns="http://www.tei-c.org/ns/1.0"><teiHeader>
		<fileDesc>
			<titleStmt>
				<title type="main">Amtsblatt des Kantons Zürich</title>
				<respStmt>
					<resp key="transcript" ref="#OCR">Transkript: OCR <date when="2024-07-12">12.07.2024</date></resp>
					<name key="editor">Staatsarchiv des Kantons Zürich</name>
				</respStmt>
			</titleStmt>
			<publicationStmt>
				<authority>Staatsarchiv des Kantons Zürich</authority>
				<date> 2024</date>
				<pubPlace>Zürich</pubPlace>
			</publicationStmt>
			<seriesStmt>
				<title>Amtsblatt des Kantons Zürich seit 1834 online</title>
				<editor>Staatsarchiv des Kantons Zürich</editor>
			</seriesStmt>
			<sourceDesc>
				<bibl>
					<title>Gründungsvertrag für die Verkehrsbetriebe der Region Unterer Zimmerberg</title><edition>Amtsblatt des Kantons Zürich seit 1834 online</edition>
					<date when="1980-02-08">08.02.1980</date><ident>StAZH ABl 1980 (S. 134-140)</ident><idno>134–140</idno><idno type="issue">9</idno><ref target="https://suche.staatsarchiv.djiktzh.ch/detail.aspx?ID=5131734"/></bibl>
			</sourceDesc>
		</fileDesc>
	</teiHeader>
	<facsimile><surface xml:id="facs_135" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0135.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_135_r13" ulx="393" uly="929" lrx="435" lry="958" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r24" ulx="60" uly="1575" lrx="241" lry="1597" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r37" ulx="60" uly="2214" lrx="250" lry="2232" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r18" ulx="302" uly="1122" lrx="1593" lry="1158" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r32" ulx="387" uly="1947" lrx="1593" lry="1980" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r27" ulx="306" uly="1662" lrx="1591" lry="1697" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r17" ulx="301" uly="1075" lrx="1591" lry="1110" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r10" ulx="454" uly="783" lrx="828" lry="812" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r5" ulx="61" uly="635" lrx="205" lry="656" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r12" ulx="452" uly="856" lrx="945" lry="888" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r39" ulx="62" uly="2270" lrx="239" lry="2291" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r31" ulx="301" uly="1877" lrx="1373" lry="1912" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r16" ulx="452" uly="1002" lrx="785" lry="1032" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r15" ulx="391" uly="1002" lrx="435" lry="1031" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r20" ulx="303" uly="1218" lrx="1585" lry="1256" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r7" ulx="395" uly="710" lrx="437" lry="741" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r11" ulx="391" uly="856" lrx="437" lry="887" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r6" ulx="391" uly="635" lrx="1016" lry="671" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r30" ulx="304" uly="1829" lrx="1589" lry="1862" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r40" ulx="387" uly="2179" lrx="1587" lry="2214" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r3" ulx="881" uly="385" lrx="998" lry="421" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r42" ulx="302" uly="2277" lrx="920" lry="2309" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r14" ulx="452" uly="929" lrx="968" lry="961" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r23" ulx="712" uly="1475" lrx="1181" lry="1507" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r34" ulx="305" uly="2043" lrx="1591" lry="2078" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r35" ulx="301" uly="2091" lrx="1352" lry="2126" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r4" ulx="847" uly="554" lrx="1042" lry="583" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r8" ulx="452" uly="710" lrx="830" lry="741" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r21" ulx="302" uly="1270" lrx="964" lry="1301" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r1" ulx="721" uly="256" lrx="1165" lry="292" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r25" ulx="389" uly="1566" lrx="1589" lry="1601" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r26" ulx="305" uly="1614" lrx="1589" lry="1649" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r2" ulx="304" uly="316" lrx="1593" lry="360" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r28" ulx="301" uly="1712" lrx="1183" lry="1746" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r36" ulx="59" uly="2183" lrx="239" lry="2203" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r29" ulx="385" uly="1779" lrx="1591" lry="1815" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r33" ulx="303" uly="1997" lrx="1593" lry="2028" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r22" ulx="392" uly="1337" lrx="1343" lry="1372" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r38" ulx="58" uly="2243" lrx="256" lry="2263" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r0" ulx="308" uly="160" lrx="373" lry="185" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r41" ulx="304" uly="2229" lrx="1591" lry="2262" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r9" ulx="391" uly="783" lrx="437" lry="812" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_135_r19" ulx="302" uly="1170" lrx="1591" lry="1207" rendition="Line" subtype="paragraph"/>
		</surface>
		<surface xml:id="facs_136" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0136.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_136_r7" ulx="143" uly="558" lrx="1419" lry="591" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r0" ulx="1354" uly="156" lrx="1416" lry="181" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r35" ulx="145" uly="1979" lrx="1429" lry="2017" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r6" ulx="225" uly="504" lrx="1420" lry="540" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r13" ulx="227" uly="864" lrx="1406" lry="901" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r4" ulx="143" uly="387" lrx="1410" lry="425" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r39" ulx="233" uly="2210" lrx="908" lry="2246" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r25" ulx="147" uly="1493" lrx="706" lry="1526" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r10" ulx="139" uly="700" lrx="1423" lry="736" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r15" ulx="144" uly="966" lrx="1079" lry="1001" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r21" ulx="139" uly="1293" lrx="1429" lry="1331" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r38" ulx="1460" uly="2210" lrx="1562" lry="2231" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r24" ulx="145" uly="1439" lrx="1424" lry="1474" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r14" ulx="141" uly="914" lrx="1430" lry="949" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r20" ulx="231" uly="1245" lrx="1429" lry="1280" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r31" ulx="143" uly="1802" lrx="1116" lry="1835" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r19" ulx="144" uly="1189" lrx="396" lry="1217" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r29" ulx="146" uly="1700" lrx="1431" lry="1737" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r37" ulx="447" uly="2133" lrx="1131" lry="2167" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r11" ulx="141" uly="750" lrx="1423" lry="786" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r30" ulx="143" uly="1747" lrx="1428" lry="1784" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r23" ulx="145" uly="1389" lrx="1429" lry="1426" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r1" ulx="497" uly="256" lrx="1071" lry="288" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r2" ulx="1447" uly="335" lrx="1604" lry="363" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r17" ulx="145" uly="1077" lrx="1429" lry="1114" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r22" ulx="141" uly="1341" lrx="1410" lry="1379" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r9" ulx="141" uly="652" lrx="1404" lry="688" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r27" ulx="144" uly="1604" lrx="1429" lry="1642" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r32" ulx="1455" uly="1885" lrx="1637" lry="1913" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r40" ulx="170" uly="2281" lrx="712" lry="2313" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r16" ulx="231" uly="1031" lrx="1425" lry="1067" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r3" ulx="227" uly="339" lrx="1416" lry="377" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r36" ulx="145" uly="2035" lrx="1158" lry="2066" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r33" ulx="231" uly="1885" lrx="1429" lry="1921" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r28" ulx="141" uly="1652" lrx="1431" lry="1689" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r12" ulx="143" uly="806" lrx="354" lry="834" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r5" ulx="143" uly="447" lrx="827" lry="477" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r18" ulx="141" uly="1122" lrx="1427" lry="1165" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r26" ulx="227" uly="1556" lrx="1429" lry="1592" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r34" ulx="145" uly="1935" lrx="1426" lry="1970" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_136_r8" ulx="141" uly="602" lrx="1428" lry="639" rendition="Line" subtype="paragraph"/>
		</surface>
		<surface xml:id="facs_137" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0137.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_137_r30" ulx="306" uly="1604" lrx="743" lry="1632" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r34" ulx="310" uly="1820" lrx="562" lry="1848" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r13" ulx="683" uly="943" lrx="1231" lry="971" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r4" ulx="402" uly="452" lrx="1600" lry="485" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r9" ulx="316" uly="693" lrx="1595" lry="727" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r36" ulx="64" uly="1947" lrx="278" lry="1966" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r17" ulx="66" uly="1104" lrx="220" lry="1122" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r18" ulx="68" uly="1135" lrx="150" lry="1152" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r7" ulx="316" uly="600" lrx="1604" lry="632" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r8" ulx="314" uly="643" lrx="1600" lry="680" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r28" ulx="314" uly="1504" lrx="1604" lry="1539" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r39" ulx="311" uly="1962" lrx="1602" lry="1997" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r31" ulx="397" uly="1672" lrx="1602" lry="1708" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r45" ulx="308" uly="2277" lrx="1596" lry="2312" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r25" ulx="311" uly="1333" lrx="550" lry="1361" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r22" ulx="400" uly="1187" lrx="1595" lry="1221" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r14" ulx="70" uly="1014" lrx="232" lry="1032" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r38" ulx="397" uly="1914" lrx="1602" lry="1948" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r33" ulx="310" uly="1770" lrx="1600" lry="1804" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r1" ulx="335" uly="252" lrx="1185" lry="284" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r35" ulx="65" uly="1916" lrx="236" lry="1936" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r6" ulx="313" uly="545" lrx="1604" lry="582" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r43" ulx="308" uly="2181" lrx="1600" lry="2214" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r12" ulx="315" uly="839" lrx="508" lry="867" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r44" ulx="306" uly="2229" lrx="1600" lry="2261" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r21" ulx="314" uly="1114" lrx="895" lry="1146" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r10" ulx="316" uly="743" lrx="1604" lry="778" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r40" ulx="310" uly="2008" lrx="1600" lry="2043" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r11" ulx="316" uly="787" lrx="1602" lry="825" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r23" ulx="312" uly="1239" lrx="1597" lry="1271" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r16" ulx="68" uly="1075" lrx="280" lry="1093" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r3" ulx="335" uly="372" lrx="1133" lry="409" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r27" ulx="312" uly="1456" lrx="1597" lry="1491" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r5" ulx="314" uly="500" lrx="1606" lry="535" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r41" ulx="310" uly="2060" lrx="1368" lry="2094" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r26" ulx="400" uly="1406" lrx="1602" lry="1441" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r0" ulx="316" uly="158" lrx="379" lry="182" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r20" ulx="314" uly="1066" lrx="1602" lry="1101" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r29" ulx="309" uly="1552" lrx="1597" lry="1587" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r32" ulx="312" uly="1725" lrx="1598" lry="1757" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r37" ulx="64" uly="1975" lrx="195" lry="1996" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r42" ulx="396" uly="2131" lrx="1597" lry="2164" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r19" ulx="404" uly="1018" lrx="1602" lry="1052" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r24" ulx="313" uly="1283" lrx="1600" lry="1318" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r15" ulx="66" uly="1043" lrx="285" lry="1064" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_137_r2" ulx="337" uly="314" lrx="783" lry="346" rendition="Line" subtype="paragraph"/>
		</surface>
		<surface xml:id="facs_138" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0138.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_138_r31" ulx="233" uly="1710" lrx="1433" lry="1744" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r7" ulx="1458" uly="585" lrx="1629" lry="606" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r33" ulx="170" uly="1827" lrx="1433" lry="1859" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r14" ulx="234" uly="881" lrx="1428" lry="914" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r18" ulx="152" uly="1075" lrx="1431" lry="1107" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r36" ulx="235" uly="2014" lrx="1219" lry="2048" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r19" ulx="147" uly="1125" lrx="1427" lry="1157" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r22" ulx="1462" uly="1312" lrx="1606" lry="1330" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r11" ulx="145" uly="718" lrx="1429" lry="751" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r27" ulx="168" uly="1497" lrx="1425" lry="1530" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r9" ulx="151" uly="602" lrx="1425" lry="635" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r28" ulx="231" uly="1543" lrx="1433" lry="1576" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r21" ulx="145" uly="1225" lrx="539" lry="1257" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r20" ulx="147" uly="1175" lrx="1433" lry="1205" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r0" ulx="1361" uly="156" lrx="1425" lry="181" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r35" ulx="170" uly="1968" lrx="1437" lry="1999" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r23" ulx="1460" uly="1341" lrx="1612" lry="1361" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r15" ulx="150" uly="931" lrx="1435" lry="963" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r3" ulx="234" uly="368" lrx="1410" lry="402" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r13" ulx="147" uly="814" lrx="460" lry="846" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r32" ulx="237" uly="1760" lrx="1433" lry="1792" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r39" ulx="231" uly="2210" lrx="593" lry="2238" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r34" ulx="170" uly="1898" lrx="708" lry="1930" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r6" ulx="1456" uly="552" lrx="1647" lry="573" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r37" ulx="168" uly="2089" lrx="945" lry="2120" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r26" ulx="233" uly="1429" lrx="1281" lry="1461" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r12" ulx="149" uly="764" lrx="1427" lry="799" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r25" ulx="168" uly="1381" lrx="1433" lry="1414" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r1" ulx="147" uly="252" lrx="1430" lry="285" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r4" ulx="147" uly="420" lrx="1422" lry="450" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r5" ulx="144" uly="466" lrx="464" lry="501" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r41" ulx="231" uly="2327" lrx="591" lry="2356" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r24" ulx="233" uly="1310" lrx="1053" lry="1342" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r10" ulx="235" uly="670" lrx="1428" lry="701" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r8" ulx="231" uly="552" lrx="1430" lry="584" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r40" ulx="168" uly="2279" lrx="1437" lry="2311" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r38" ulx="166" uly="2158" lrx="1431" lry="2190" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r2" ulx="147" uly="308" lrx="472" lry="334" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r29" ulx="231" uly="1593" lrx="1110" lry="1626" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r16" ulx="147" uly="981" lrx="1436" lry="1012" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r17" ulx="147" uly="1029" lrx="1431" lry="1060" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_138_r30" ulx="168" uly="1662" lrx="1433" lry="1694" rendition="Line" subtype="paragraph"/>
		</surface>
		<surface xml:id="facs_139" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0139.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_139_r11" ulx="556" uly="952" lrx="1341" lry="987" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r25" ulx="312" uly="1477" lrx="891" lry="1506" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r4" ulx="397" uly="420" lrx="1597" lry="454" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r6" ulx="722" uly="585" lrx="1177" lry="614" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r39" ulx="310" uly="2229" lrx="1600" lry="2261" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r36" ulx="310" uly="2060" lrx="1600" lry="2094" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r26" ulx="791" uly="1612" lrx="1112" lry="1641" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r0" ulx="312" uly="158" lrx="372" lry="183" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r35" ulx="308" uly="2012" lrx="1596" lry="2046" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r17" ulx="308" uly="1139" lrx="1595" lry="1173" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r30" ulx="305" uly="1752" lrx="1592" lry="1782" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r28" ulx="68" uly="1733" lrx="258" lry="1751" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r2" ulx="395" uly="302" lrx="1141" lry="337" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r34" ulx="395" uly="1962" lrx="1597" lry="1996" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r10" ulx="306" uly="839" lrx="643" lry="867" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r29" ulx="395" uly="1702" lrx="1597" lry="1732" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r32" ulx="312" uly="1843" lrx="1600" lry="1877" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r16" ulx="308" uly="1091" lrx="1596" lry="1125" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r20" ulx="70" uly="1331" lrx="210" lry="1350" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r23" ulx="308" uly="1381" lrx="1592" lry="1412" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r40" ulx="307" uly="2283" lrx="481" lry="2310" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r38" ulx="391" uly="2179" lrx="1593" lry="2214" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r13" ulx="64" uly="1072" lrx="235" lry="1093" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r5" ulx="395" uly="466" lrx="820" lry="498" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r37" ulx="310" uly="2108" lrx="935" lry="2140" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r7" ulx="395" uly="670" lrx="1598" lry="704" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r18" ulx="306" uly="1187" lrx="1598" lry="1223" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r15" ulx="393" uly="1043" lrx="1597" lry="1076" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r9" ulx="397" uly="787" lrx="1589" lry="821" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r33" ulx="308" uly="1895" lrx="693" lry="1926" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r22" ulx="393" uly="1331" lrx="1596" lry="1363" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r1" ulx="329" uly="256" lrx="1595" lry="288" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r8" ulx="306" uly="720" lrx="1164" lry="754" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r21" ulx="68" uly="1362" lrx="152" lry="1380" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r3" ulx="329" uly="370" lrx="1595" lry="405" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r31" ulx="307" uly="1795" lrx="1591" lry="1828" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r19" ulx="306" uly="1239" lrx="1212" lry="1271" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r14" ulx="68" uly="1100" lrx="147" lry="1119" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r12" ulx="68" uly="1043" lrx="233" lry="1062" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r24" ulx="310" uly="1429" lrx="1595" lry="1460" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_139_r27" ulx="66" uly="1697" lrx="250" lry="1718" rendition="Line" subtype="paragraph"/>
		</surface>
		<surface xml:id="facs_140" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0140.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_140_r22" ulx="1443" uly="968" lrx="1656" lry="994" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r18" ulx="220" uly="793" lrx="1409" lry="824" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r17" ulx="1445" uly="825" lrx="1635" lry="844" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r43" ulx="129" uly="1777" lrx="762" lry="1809" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r42" ulx="133" uly="1727" lrx="1420" lry="1760" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r0" ulx="1352" uly="158" lrx="1412" lry="182" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r29" ulx="135" uly="1114" lrx="1006" lry="1146" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r27" ulx="132" uly="1014" lrx="1410" lry="1048" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r30" ulx="220" uly="1177" lrx="1415" lry="1209" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r6" ulx="133" uly="402" lrx="987" lry="432" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r31" ulx="133" uly="1222" lrx="1424" lry="1257" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r37" ulx="131" uly="1462" lrx="497" lry="1491" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r1" ulx="1439" uly="249" lrx="1612" lry="273" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r11" ulx="985" uly="512" lrx="1162" lry="543" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r21" ulx="666" uly="889" lrx="914" lry="920" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r23" ulx="1445" uly="997" lrx="1652" lry="1018" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r39" ulx="133" uly="1568" lrx="1146" lry="1601" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r28" ulx="135" uly="1064" lrx="1425" lry="1098" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r38" ulx="220" uly="1516" lrx="1420" lry="1551" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r25" ulx="1445" uly="1058" lrx="1585" lry="1080" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r44" ulx="1441" uly="1866" lrx="1633" lry="1893" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r7" ulx="216" uly="464" lrx="843" lry="496" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r13" ulx="135" uly="560" lrx="1413" lry="592" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r3" ulx="222" uly="252" lrx="1417" lry="285" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r49" ulx="129" uly="2064" lrx="256" lry="2092" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r40" ulx="216" uly="1629" lrx="1421" lry="1662" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r34" ulx="1447" uly="1395" lrx="1631" lry="1416" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r24" ulx="1441" uly="1022" lrx="1615" lry="1046" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r9" ulx="1064" uly="462" lrx="1415" lry="493" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r4" ulx="133" uly="304" lrx="1417" lry="336" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r35" ulx="218" uly="1358" lrx="1425" lry="1392" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r8" ulx="868" uly="464" lrx="1059" lry="495" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r26" ulx="220" uly="966" lrx="1417" lry="998" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r32" ulx="133" uly="1272" lrx="1419" lry="1306" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r33" ulx="1443" uly="1360" lrx="1592" lry="1387" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r45" ulx="218" uly="1864" lrx="1417" lry="1896" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r12" ulx="1170" uly="512" lrx="1415" lry="541" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r20" ulx="133" uly="891" lrx="641" lry="921" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r47" ulx="131" uly="1958" lrx="1420" lry="1992" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r5" ulx="134" uly="350" lrx="1417" lry="383" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r15" ulx="600" uly="716" lrx="962" lry="744" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r48" ulx="133" uly="2006" lrx="1416" lry="2042" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r2" ulx="1445" uly="285" lrx="1529" lry="302" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r10" ulx="131" uly="514" lrx="958" lry="545" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r14" ulx="135" uly="612" lrx="585" lry="641" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r19" ulx="135" uly="841" lrx="1412" lry="873" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r46" ulx="133" uly="1912" lrx="1417" lry="1944" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r41" ulx="135" uly="1683" lrx="1415" lry="1712" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r36" ulx="133" uly="1406" lrx="1412" lry="1441" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_140_r16" ulx="1445" uly="793" lrx="1631" lry="819" rendition="Line" subtype="paragraph"/>
		</surface>
		<surface xml:id="facs_141" uly="0" ulx="0" lry="2598" lrx="1748">
			<graphic url="Abl_1980_1__S__0141.tif" width="1748px" height="2598px"/>
			<zone xml:id="facs_141_r33" ulx="410" uly="2166" lrx="1614" lry="2198" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r21" ulx="435" uly="1383" lrx="852" lry="1412" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r9" ulx="541" uly="645" lrx="1608" lry="679" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r4" ulx="326" uly="462" lrx="1120" lry="494" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r28" ulx="870" uly="1808" lrx="1066" lry="1837" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r23" ulx="411" uly="1529" lrx="1610" lry="1565" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r30" ulx="410" uly="1962" lrx="1613" lry="1994" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r19" ulx="408" uly="1245" lrx="847" lry="1275" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r15" ulx="870" uly="975" lrx="1068" lry="1004" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r20" ulx="433" uly="1312" lrx="1185" lry="1344" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r24" ulx="322" uly="1579" lrx="1610" lry="1612" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r12" ulx="539" uly="762" lrx="1589" lry="796" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r3" ulx="412" uly="414" lrx="1608" lry="448" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r6" ulx="536" uly="527" lrx="1612" lry="560" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r17" ulx="416" uly="1125" lrx="1610" lry="1158" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r27" ulx="324" uly="1727" lrx="487" lry="1755" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r2" ulx="833" uly="329" lrx="1110" lry="361" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r31" ulx="322" uly="2006" lrx="1612" lry="2042" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r29" ulx="827" uly="1877" lrx="1108" lry="1909" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r25" ulx="329" uly="1629" lrx="1612" lry="1662" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r8" ulx="325" uly="645" lrx="506" lry="676" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r34" ulx="320" uly="2214" lrx="1189" lry="2248" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r32" ulx="411" uly="2075" lrx="1008" lry="2107" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r10" ulx="544" uly="700" lrx="610" lry="723" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r22" ulx="326" uly="1468" lrx="806" lry="1499" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r1" ulx="872" uly="260" lrx="1066" lry="291" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r18" ulx="324" uly="1175" lrx="1390" lry="1208" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r16" ulx="835" uly="1043" lrx="1112" lry="1074" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r5" ulx="320" uly="529" lrx="508" lry="560" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r26" ulx="325" uly="1677" lrx="1606" lry="1710" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r14" ulx="545" uly="860" lrx="1587" lry="894" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r0" ulx="333" uly="162" lrx="393" lry="186" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r11" ulx="322" uly="764" lrx="506" lry="792" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r7" ulx="541" uly="577" lrx="939" lry="610" rendition="Line" subtype="paragraph"/>
			<zone xml:id="facs_141_r13" ulx="543" uly="812" lrx="1613" lry="844" rendition="Line" subtype="paragraph"/>
		</surface>
	</facsimile><text><body><pb n="134" facs="#facs_135"/>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_135_r4"/>I. Ingress</head>
				<p>
					<add type="marginal" place="left">
						<lb facs="#facs_135_r5"/>Gründung</add>
				</p>
				<p>
					<lb facs="#facs_135_r6"/>§ 1. Die folgenden Beteiligten</p>
				<table>
					<row>
						<cell>
							<lb facs="#facs_135_r7"/>1.</cell>
						<cell>
							<lb facs="#facs_135_r8"/>der Kanton <placeName n="a5ddb64d-059e-48f7-9db6-4d059ee8f791">Zürich</placeName></cell>
					</row>
					<row>
						<cell>
							<lb facs="#facs_135_r9"/>2.</cell>
						<cell>
							<lb facs="#facs_135_r10"/>die Stadt <placeName n="cd0b3bca-3ea6-416b-8b3b-ca3ea6c16bfe">Adliswil</placeName></cell>
					</row>
					<row>
						<cell>
							<lb facs="#facs_135_r11"/>3.</cell>
						<cell>
							<lb facs="#facs_135_r12"/>die Gemeinde <placeName n="97d155da-4b5f-4cdc-9155-da4b5ffcdc15">Kilchberg</placeName></cell>
					</row>
					<row>
						<cell>
							<lb facs="#facs_135_r13"/>4.</cell>
						<cell>
							<lb facs="#facs_135_r14"/>die Gemeinde <placeName n="fdc817b1-c431-4aad-8817-b1c4313aade3">Rüschlikon</placeName></cell>
					</row>
					<row>
						<cell>
							<lb facs="#facs_135_r15"/>5.</cell>
						<cell>
							<lb facs="#facs_135_r16"/>die Stadt <placeName n="c3bb6527-1ba5-44a6-bb65-271ba574a69a">Zürich</placeName></cell>
					</row>
				</table>
				<p>
					<lb facs="#facs_135_r17"/>gründen hiermit unter Vorbehalt der Änderung des Kreises<lb facs="#facs_135_r18"/>der Beteiligten die Unternehmung «<orgName n="f9afe839-23e0-4b0e-afe8-3923e03b0e18">Verkehrsbetriebe der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg</orgName> (<orgName n="473cba5f-8e66-40b9-bcba-5f8e66b0b9f5">VRUZ</orgName>)» als regionalen Ver<lb break="no" facs="#facs_135_r20"/>kehrsbetrieb im Sinne der §§ 4-13 des Gesetzes über den<lb facs="#facs_135_r21"/>regionalen öffentlichen Verkehr.</p>
				<p>
					<lb facs="#facs_135_r22"/>Die Unternehmung hat ihren Sitz in <placeName n="16809273-5651-426c-8092-735651c26cb9">Kilchberg.</placeName></p>
			</div>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_135_r23"/>II. Zweck und Aufgabe</head>
				<p>
					<add type="marginal" place="left">
						<lb facs="#facs_135_r24"/>Allgemeines</add>
				</p>
				<p>
					<lb facs="#facs_135_r25"/>§2. Die <orgName n="a6e49c77-b6f7-41a8-a49c-77b6f741a84a">VRUZ</orgName> sorgen für eine gute Bedienung der Re<lb break="no" facs="#facs_135_r26"/>gion <placeName n="4827755e-a4e4-4cbb-a775-5ea4e48cbb61">Unterer Zimmerberg</placeName> und jeder beteiligten Gemeinde<lb facs="#facs_135_r27"/>mit öffentlichen Verkehrsmitteln durch den Ausbau des Bus<lb break="no" facs="#facs_135_r28"/>netzes und der dazu nötigen Einrichtungen.</p>
				<p>
					<lb facs="#facs_135_r29"/>Die <orgName n="9d7f7d4b-3970-4697-bf7d-4b3970369737">VRUZ</orgName> erlassen eine für ihr ganzes Verkehrsnetz ge<lb break="no" facs="#facs_135_r30"/>mäss Anhang 1 gültige Taxordnung, wobei die <placeName n="e8b74d0c-bf10-4ddc-b74d-0cbf10bddc8c" type="deriv">Zürcher</placeName> Stamm<lb break="no" facs="#facs_135_r31"/>linien der <orgName n="b397e0de-07da-4738-97e0-de07da87389e">VBZ</orgName> gemäss Anhang 3 ausgenommen sind.</p>
				<p>
					<lb facs="#facs_135_r32"/>Sie streben den Abschluss von Tarif- und Verbund<lb break="no" facs="#facs_135_r33"/>abkommen mit den <orgName n="331d1ae0-5792-4602-9d1a-e05792260237">VBZ</orgName>, der <orgName n="8aef8c82-3bc7-4d1a-af8c-823bc7ad1a45">SZU</orgName> und den <orgName n="43f5676b-7d87-4f75-b567-6b7d87af7551">SBB</orgName> sowie mit<lb facs="#facs_135_r34"/>anderen Verkehrsunternehmungen und die gegenseitige Ab<lb break="no" facs="#facs_135_r35"/>stimmung der Verkehrsnetze und der Fahrpläne an.</p>
				<p>
					<add type="marginal" place="left">
						<lb facs="#facs_135_r36"/>Übertragung<lb facs="#facs_135_r37"/>des Betriebes<lb facs="#facs_135_r38"/>an Transport<lb break="no" facs="#facs_135_r39"/>beauftragten</add>
				</p>
				<p>
					<lb facs="#facs_135_r40"/>§3. Die <orgName n="3fb849b7-f897-474f-b849-b7f897474f01">VRUZ</orgName> betrauen einen Transportbeauftragten<lb facs="#facs_135_r41"/>mit dem Betrieb ihres Verkehrsnetzes. Sie können eine öffent<lb break="no" facs="#facs_135_r42"/>liche Submission durchführen.</p>
			</div>
			<pb n="135" facs="#facs_136"/>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_136_r1"/>III. Beteiligungsverhältnisse</head>
				<p>
					<add type="marginal" place="right">
						<lb facs="#facs_136_r2"/>Grundsatz</add>
				</p>
				<p>
					<lb facs="#facs_136_r3"/>§ 4. Die Beteiligung der Vorortsgemeinden am allfälligen<lb facs="#facs_136_r4"/>Betriebsdefizit richtet sich nach dem Verteilschlüssel im<lb facs="#facs_136_r5"/>Anhang 2 zum Gründungsvertrag.</p>
				<p>
					<lb facs="#facs_136_r6"/>Die Stadt <placeName n="f968ca3e-0feb-4ec8-a8ca-3e0feb1ec83a">Zürich</placeName> beteiligt sich entsprechend dem Betriebs<lb break="no" facs="#facs_136_r7"/>defizit, welches auf die im Anhang 3 zum Gründungsvertrag<lb facs="#facs_136_r8"/>aufgeführten Bus -, Trolleybus- und Tramlinien entfällt, wel<lb break="no" facs="#facs_136_r9"/>che die Region <placeName n="0ef30930-8296-42d8-b309-308296d2d872">Unterer Zimmerberg</placeName> mit dem Stadtzentrum<lb facs="#facs_136_r10"/>verbinden. Dabei werden nur die Kurse berücksichtigt, welche<lb facs="#facs_136_r11"/>den Anschluss an Kurse aus der Region Unterer Zimmerberg<lb facs="#facs_136_r12"/>herstellen.</p>
				<p>
					<lb facs="#facs_136_r13"/>Für Leistungen, welche die <orgName n="78df796c-89a0-43f9-9f79-6c89a0b3f981">VRUZ</orgName> zu ihren Lasten im<lb facs="#facs_136_r14"/>Bereich des Stadttarifes erbringen, haben sie Anspruch auf<lb facs="#facs_136_r15"/>eine angemessene Abgeltung der Stadt <placeName n="6c0e08a8-ca62-4c88-8e08-a8ca621c8804">Zürich.</placeName></p>
				<p>
					<lb facs="#facs_136_r16"/>Der <orgName n="0b9756b6-01b3-4f23-9756-b601b3ef2384">Kanton</orgName> beteiligt sich am Betriebsdefizit nach Abzug<lb facs="#facs_136_r17"/>allfälliger Beiträge Dritter mit 30 Prozent; eine höhere Be<lb break="no" facs="#facs_136_r18"/>teiligung bei den vom <orgName n="c3cb4b93-f8fa-49bb-8b4b-93f8fa39bbfa">Bund</orgName> subventionierten Linien bleibt<lb facs="#facs_136_r19"/>vorbehalten.</p>
				<p>
					<lb facs="#facs_136_r20"/>In bezug auf die Gemeinden der Region <placeName n="e387820b-5501-4ba4-8782-0b5501dba42d">Unterer Zimmer<lb break="no" facs="#facs_136_r21"/>berg</placeName> wird das Betriebsdefizit nur soweit anerkannt, als an<lb break="no" facs="#facs_136_r22"/>gemessene Tarife erhoben werden. Der Beteiligung an dem<lb facs="#facs_136_r23"/>von der Stadt <placeName n="da17a764-b353-4bf0-97a7-64b3537bf03a">Zürich</placeName> zu tragenden Betriebsdefizit wird eine<lb facs="#facs_136_r24"/>Kostendeckung der <orgName n="de276f46-fa10-4567-a76f-46fa10d56744">Verkehrsbetriebe der Stadt <placeName n="fd17cbd7-2496-4ab1-97cb-d724961ab132">Zürich</placeName></orgName> von<lb facs="#facs_136_r25"/>70 Prozent zugrunde gelegt.</p>
				<p>
					<lb facs="#facs_136_r26"/>Jede Vertragsgemeinde ist für ihr Gebiet verpflichtet,<lb facs="#facs_136_r27"/>genügende Haltestellenanlagen, wie Ausbuchtungen, Warte<lb break="no" facs="#facs_136_r28"/>hallen, Anschluss von Werkleitungen usw ., unentgeltlich zur<lb facs="#facs_136_r29"/>Verfügung zu stellen. Die beteiligten Gemeinwesen sind bei<lb facs="#facs_136_r30"/>Schneefall für rechtzeitiges Räumen, Salzen und Splitten der<lb facs="#facs_136_r31"/>betreffenden Strassen auf ihrem Gebiet besorgt.</p>
				<p>
					<add type="marginal" place="right">
						<lb facs="#facs_136_r32"/>Änderungen</add>
				</p>
				<p>
					<lb facs="#facs_136_r33"/>§5. Bei einer Änderung des Verkehrsnetzes, einer<lb facs="#facs_136_r34"/>wesentlichen Änderung in der Bedienung oder bei anderen<lb facs="#facs_136_r35"/>Verschiebungen in den Grundlagen des Verteilschlüssels ist<lb facs="#facs_136_r36"/>dieser den veränderten Verhältnissen anzupassen.</p>
			</div>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_136_r37"/>IV. Organisation und Verwaltung</head>
				<p>
					<add type="marginal" place="right">
						<lb facs="#facs_136_r38"/>Organe</add>
				</p>
				<p>
					<lb facs="#facs_136_r39"/>§ 6. Die Organe der <orgName n="617e82f7-7e4b-402e-be82-f77e4bb02e99">VRUZ</orgName> sind:</p>
				<p>
					<lb facs="#facs_136_r40"/>a) die Betriebskommission</p>
				<pb n="136" facs="#facs_137"/>
				<p>
					<lb facs="#facs_137_r1"/>b) der Ausschuss der Betriebskommission</p>
				<p>
					<lb facs="#facs_137_r2"/>c) die Betriebsleitung</p>
				<p>
					<lb facs="#facs_137_r3"/>d) die <orgName n="5c7d0993-d237-4989-bd09-93d23769897b">Rechnungsprüfungskommission.</orgName></p>
				<p>
					<lb facs="#facs_137_r4"/>Wenn einzelne Geschäfte die Befugnisse der Betriebs<lb break="no" facs="#facs_137_r5"/>kommission übersteigen, hat diese den Beteiligten einen An<lb break="no" facs="#facs_137_r6"/>trag zu unterbreiten. Der Antrag der Betriebskommission gilt<lb facs="#facs_137_r7"/>als angenommen, wenn ihm mindestens vier Fünftel aller<lb facs="#facs_137_r8"/>Beteiligten und auf alle Fälle die direkt betroffenen Gemein<lb break="no" facs="#facs_137_r9"/>den zustimmen. Solche Beschlüsse sind für alle Beteiligten<lb facs="#facs_137_r10"/>verbindlich. Vorbehalten bleiben die Vorschriften gemäss<lb facs="#facs_137_r11"/>§§ 17 und 18 für die Änderung dieses Vertrages und seiner<lb facs="#facs_137_r12"/>Anhänge.</p>
				<div type="subtitle" n="2">
					<head type="subtitle" n="2">
						<lb facs="#facs_137_r13"/>A. Die Betriebskommission</head>
					<p>
						<add type="marginal" place="left">
							<lb facs="#facs_137_r14"/>Zusammen<lb break="no" facs="#facs_137_r15"/>setzung, Wahl,<lb facs="#facs_137_r16"/>Konstituierung<lb facs="#facs_137_r17"/>und Amts<lb break="no" facs="#facs_137_r18"/>dauer</add>
					</p>
					<p>
						<lb facs="#facs_137_r19"/>§7. Die beteiligten Gemeinwesen ordnen je einen Ver<lb break="no" facs="#facs_137_r20"/>treter in die Betriebskommission ab. Sie bestimmen gleich<lb break="no" facs="#facs_137_r21"/>zeitig je einen Stellvertreter.</p>
					<p>
						<lb facs="#facs_137_r22"/>Die Mitglieder der Betriebskommission können sich an<lb facs="#facs_137_r23"/>den Sitzungen durch ihre Stellvertreter vertreten lassen. Die<lb facs="#facs_137_r24"/>Stellvertreter können an jeder Sitzung mit beratender Stimme<lb facs="#facs_137_r25"/>teilnehmen.</p>
					<p>
						<lb facs="#facs_137_r26"/>Die Wahl der Mitglieder und der Stellvertreter erfolgt auf<lb facs="#facs_137_r27"/>die Amtsdauer der Gemeindebehörden. Sie richtet sich nach<lb facs="#facs_137_r28"/>den Vorschriften der entsprechenden Gemeindeordnungen.<lb facs="#facs_137_r29"/>Der Regierungsrat bestimmt das Mitglied des <orgName n="578913b7-1475-4828-8913-b71475d82829">Kantons</orgName> und<lb facs="#facs_137_r30"/>seinen Stellvertreter.</p>
					<p>
						<lb facs="#facs_137_r31"/>Die Betriebskommission konstituiert sich selbst. Sie be<lb break="no" facs="#facs_137_r32"/>zeichnet einen Sekretär, der nicht Mitglied der Betriebs<lb break="no" facs="#facs_137_r33"/>kommission zu sein braucht und in diesem Falle beratende<lb facs="#facs_137_r34"/>Stimme hat.</p>
					<p>
						<add type="marginal" place="left">
							<lb facs="#facs_137_r35"/>Einberufung<lb facs="#facs_137_r36"/>und Beschluss<lb break="no" facs="#facs_137_r37"/>fähigkeit</add>
					</p>
					<p>
						<lb facs="#facs_137_r38"/>§ 8. Die Betriebskommission versammelt sich auf Ein<lb break="no" facs="#facs_137_r39"/>ladung des Präsidenten oder bei dessen Verhinderung des<lb facs="#facs_137_r40"/>Vizepräsidenten so oft es die Geschäfte erfordern. Sie muss<lb facs="#facs_137_r41"/>einberufen werden, wenn ein Mitglied dies verlangt.</p>
					<p>
						<lb facs="#facs_137_r42"/>Sie ist beschlussfähig, wenn die Mehrheit aller Mitglieder<lb facs="#facs_137_r43"/>oder deren Stellvertreter anwesend sind. Ihre Beschlüsse fasst<lb facs="#facs_137_r44"/>sie unter Vorbehalt der §§ 17 und 18 mit einfachem Mehr der<lb facs="#facs_137_r45"/>anwesenden stimmberechtigten Mitglieder. Jedem Mitglied</p>
					<pb n="137" facs="#facs_138"/>
					<p>
						<lb facs="#facs_138_r1"/>oder bei dessen Verhinderung seinem Stellvertreter steht nur<lb facs="#facs_138_r2"/>eine Stimme zu.</p>
					<p>
						<lb facs="#facs_138_r3"/>Über die Beschlüsse wird ein Protokoll geführt, das vom<lb facs="#facs_138_r4"/>Sekretär zu unterzeichnen und von der Betriebskommission zu<lb facs="#facs_138_r5"/>genehmigen ist.</p>
					<p>
						<add type="marginal" place="right">
							<lb facs="#facs_138_r6"/>Befugnisse im<lb facs="#facs_138_r7"/>allgemeinen</add>
					</p>
					<p>
						<lb facs="#facs_138_r8"/>§ 9. Die Betriebskommission verwaltet und vertritt die<lb facs="#facs_138_r9"/>Unternehmung, soweit nicht ein anderes Organ zuständig ist.</p>
					<p>
						<lb facs="#facs_138_r10"/>Sie kann aus ihrer Mitte einen Ausschuss zur Leitung und<lb facs="#facs_138_r11"/>Beaufsichtigung der laufenden Geschäfte bezeichnen und setzt<lb facs="#facs_138_r12"/>dessen Befugnisse und Obliegenheiten in einem besonderen<lb facs="#facs_138_r13"/>Reglement fest.</p>
					<p>
						<lb facs="#facs_138_r14"/>Die Betriebskommission wählt die Betriebsleitung und<lb facs="#facs_138_r15"/>bestimmt deren Aufgaben und Zuständigkeit in einem Regle<lb break="no" facs="#facs_138_r16"/>ment. Ferner bezeichnet sie diejenigen Personen aus ihrer<lb facs="#facs_138_r17"/>Mitte oder ausserhalb derselben, denen die rechtsverbindliche<lb facs="#facs_138_r18"/>Unterschrift für die Unternehmung zusteht. Die rechtsver<lb break="no" facs="#facs_138_r19"/>bindliche Vertretung der Unternehmung kann in allen Fällen<lb facs="#facs_138_r20"/>nur durch kollektive Zeichnung zweier Unterschriftsbe<lb break="no" facs="#facs_138_r21"/>rechtigter erfolgen.</p>
					<p>
						<add type="marginal" place="right">
							<lb facs="#facs_138_r22"/>Besondere<lb facs="#facs_138_r23"/>Befugnisse</add>
					</p>
					<p>
						<lb facs="#facs_138_r24"/>§ 10. Der Betriebskommission steht zu:</p>
					<p>
						<lb facs="#facs_138_r25"/>a) Antragstellung an die Beteiligten über Geschäfte, welche<lb facs="#facs_138_r26"/>die Befugnisse der Betriebskommission übersteigen.</p>
					<p>
						<lb facs="#facs_138_r27"/>b) Festsetzung des Voranschlages und die Bewilligung von<lb facs="#facs_138_r28"/>Nachtragskrediten bis zum Betrag von Fr. 100 000 für das<lb facs="#facs_138_r29"/>bestehende Streckennetz gemäss Anhang 1.</p>
					<p>
						<lb facs="#facs_138_r30"/>c) Anordnungen, die neue, durch Mehrleistungen bedingte,<lb facs="#facs_138_r31"/>jährlich wiederkehrende Ausgaben bis zu Fr. 50 000 pro<lb facs="#facs_138_r32"/>Jahr oder entsprechende Einnahmenausfälle verursachen.</p>
					<p>
						<lb facs="#facs_138_r33"/>d) Abnahme der Jahresrechnung und des Geschäftsberichtes.</p>
					<p>
						<lb facs="#facs_138_r34"/>e) Erlass der Taxordnung.</p>
					<p>
						<lb facs="#facs_138_r35"/>f) Abschluss von Abkommen über Tarif- und Verkehrsver<lb break="no" facs="#facs_138_r36"/>bunde sowie über Verkehrsleistungen für Dritte.</p>
					<p>
						<lb facs="#facs_138_r37"/>g) Aufsicht über die Betriebsführung.</p>
					<p>
						<lb facs="#facs_138_r38"/>h) Abschluss von Verträgen über den Betrieb des Verkehrs<lb break="no" facs="#facs_138_r39"/>netzes der <orgName n="43d82f62-0982-4f73-982f-6209823f73a9">VRUZ.</orgName></p>
					<p>
						<lb facs="#facs_138_r40"/>i) Antragstellung an den <orgName n="37fc05f3-379a-47a7-bc05-f3379ac7a7d9">Regierungsrat</orgName> auf Anschluss wei<lb break="no" facs="#facs_138_r41"/>terer Gemeinden.</p>
					<pb n="138" facs="#facs_139"/>
					<p>
						<lb facs="#facs_139_r1"/>k) Erlass der eigenen Geschäftsordnung sowie von Regle<lb break="no" facs="#facs_139_r2"/>menten von allgemeiner Wichtigkeit.</p>
					<p>
						<lb facs="#facs_139_r3"/>e) Festsetzung der Tag- und Sitzungsgelder für ihre Sitzun<lb break="no" facs="#facs_139_r4"/>gen sowie die Vergütungen an den Sekretär und die<lb facs="#facs_139_r5"/>Rechnungsrevisoren.</p>
				</div>
				<div type="subtitle" n="2">
					<head type="subtitle" n="2">
						<lb facs="#facs_139_r6"/>B. Die Betriebsleitung</head>
					<p>
						<lb facs="#facs_139_r7"/>§ 11. Die Betriebsleitung besorgt die Leitung der Unter<lb break="no" facs="#facs_139_r8"/>nehmung nach Massgabe des Reglementes.</p>
					<p>
						<lb facs="#facs_139_r9"/>In der Betriebskommission hat die Betriebsleitung be<lb break="no" facs="#facs_139_r10"/>ratende Stimme.</p>
				</div>
				<div type="subtitle" n="2">
					<head type="subtitle" n="2">
						<lb facs="#facs_139_r11"/>C. Die <orgName n="aee2b33c-69bb-45de-a2b3-3c69bb85debc">Rechnungsprüfungskommission</orgName></head>
					<p>
						<add type="marginal" place="left">
							<lb facs="#facs_139_r12"/>Zusammen<lb break="no" facs="#facs_139_r13"/>setzung und<lb facs="#facs_139_r14"/>Wahl</add>
					</p>
					<p>
						<lb facs="#facs_139_r15"/>§ 12. Die beteiligten Gemeinden bestimmen je ein Mit<lb break="no" facs="#facs_139_r16"/>glied ihrer Rechnungsprüfungskommission als Rechnungsrevi<lb break="no" facs="#facs_139_r17"/>sor auf die Amtsdauer der Gemeindebehörden. Ein Rechnungs<lb break="no" facs="#facs_139_r18"/>revisor wird vom <orgName n="984600e8-028b-47df-8600-e8028b77dfc4">Regierungsrat</orgName> bestimmt. Die <orgName n="9cf28461-ae29-4588-b284-61ae29858887">Rechnungs<lb break="no" facs="#facs_139_r19"/>prüfungskommission</orgName> konstituiert sich selbst.</p>
					<p>
						<add type="marginal" place="left">
							<lb facs="#facs_139_r20"/>Obliegen<lb break="no" facs="#facs_139_r21"/>heiten</add>
					</p>
					<p>
						<lb facs="#facs_139_r22"/>§ 13. Die <orgName n="46163b5c-a3ef-428f-963b-5ca3eff28fea">Rechnungsprüfungskommission</orgName> prüft die Vor<lb break="no" facs="#facs_139_r23"/>anschläge und die Rechnungen der Unternehmung im Sinne<lb facs="#facs_139_r24"/>der Bestimmungen des Gemeindegesetzes. Sie erstattet der<lb facs="#facs_139_r25"/>Betriebskommission Bericht.</p>
				</div>
			</div>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_139_r26"/>V. Finanzierung</head>
				<p>
					<add type="marginal" place="left">
						<lb facs="#facs_139_r27"/>Finanzierung<lb facs="#facs_139_r28"/>des Betriebes</add>
				</p>
				<p>
					<lb facs="#facs_139_r29"/>§ 14. Reichen die Einnahmen trotz angemessener Be<lb break="no" facs="#facs_139_r30"/>nützungstaxen zur Deckung der Ausgaben nicht aus, so sind<lb facs="#facs_139_r31"/>eintretende Defizite entsprechend dem Verteilschlüssel durch<lb facs="#facs_139_r32"/>die Beteiligten jährlich innerhalb 30 Tagen nach Rechnungs<lb break="no" facs="#facs_139_r33"/>stellung zu decken.</p>
				<p>
					<lb facs="#facs_139_r34"/>Zur Deckung der laufenden Verbindlichkeiten fordert die<lb facs="#facs_139_r35"/>rechnungsführende Gemeinde von den Beteiligten im Laufe<lb facs="#facs_139_r36"/>des Rechnungsjahres Vorschüsse ein. Auf Einberufung eines<lb facs="#facs_139_r37"/>Grundkapitals wird verzichtet.</p>
				<p>
					<lb facs="#facs_139_r38"/>Über die Verwendung eines allfälligen Reingewinnes nach<lb facs="#facs_139_r39"/>Deckung aller Verbindlichkeiten beschliesst die Betriebskom<lb break="no" facs="#facs_139_r40"/>mission.</p>
				<pb n="139" facs="#facs_140"/>
				<p>
					<add type="marginal" place="right">
						<lb facs="#facs_140_r1"/><orgName n="e445a5d1-7906-49e0-85a5-d1790679e002">Rechnungs<lb break="no" facs="#facs_140_r2"/>wesen</orgName>wesen</add>
				</p>
				<p>
					<lb facs="#facs_140_r3"/>§ 15. Für das <orgName n="718bc71e-d6bd-4c35-8bc7-1ed6bd7c3548">Rechnungswesen</orgName> sind unter Vorbehalt der<lb facs="#facs_140_r4"/>entsprechenden Bestimmungen des <orgName n="7268faf2-48cc-43be-a8fa-f248cc73be90" type="deriv">eidgenössischen</orgName> Rechts<lb facs="#facs_140_r5"/>und der Rechnungsvorschriften des Bundesamtes für Verkehr<lb facs="#facs_140_r6"/>die vorstehenden Grundsätze massgebend.</p>
				<p>
					<lb facs="#facs_140_r7"/>Das Rechnungsjahr dauert vom <date when="--01-01">1. Januar</date> bis 31. Dezember.<lb facs="#facs_140_r10"/>Die Jahresrechnung ist spätestens bis am <date when="--03-31">31. März</date> abzuschlies<lb break="no" facs="#facs_140_r13"/>sen und allen Beteiligten mit dem Bericht der Rechnungs<lb break="no" facs="#facs_140_r14"/>revisoren zuzustellen.</p>
				<div type="subtitle" n="2">
					<head type="subtitle" n="2">
						<lb facs="#facs_140_r15"/>VI. Verschiedenes</head>
					<p>
						<add type="marginal" place="right">
							<lb facs="#facs_140_r16"/>Aufsicht und<lb facs="#facs_140_r17"/>Rechtsschutz</add>
					</p>
					<p>
						<lb facs="#facs_140_r18"/>§ 16. Aufsicht und Rechtsschutz richten sich nach den<lb facs="#facs_140_r19"/>§§ 11 bis 13 des <orgName n="f35ac762-30ef-4f59-9ac7-6230ef2f5914" type="deriv">kantonalen</orgName> Gesetzes über den regionalen<lb facs="#facs_140_r20"/>öffentlichen Verkehr vom <date when="1972-06-04">4. Juni 1972.</date></p>
					<p>
						<add type="marginal" place="right">
							<lb facs="#facs_140_r22"/>Änderung und<lb facs="#facs_140_r23"/>Auflösung des<lb facs="#facs_140_r24"/>Gründungs<lb break="no" facs="#facs_140_r25"/>vertrages</add>
					</p>
					<p>
						<lb facs="#facs_140_r26"/>§ 17. Für die Änderung dieses Gründungsvertrages sowie<lb facs="#facs_140_r27"/>für dessen Auflösung ist die Zustimmung jeder der beteiligten<lb facs="#facs_140_r28"/>Gemeinden und der für die Genehmigung des Vertrages zu<lb break="no" facs="#facs_140_r29"/>ständigen kantonalen Behörden notwendig.</p>
					<p>
						<lb facs="#facs_140_r30"/>Beschlüsse über die Aufnahme von weitern Beteiligten<lb facs="#facs_140_r31"/>und die Entlassung bisheriger Beteiligter bedürfen der Zu<lb break="no" facs="#facs_140_r32"/>stimmung von zwei Dritteln der beteiligten Vorortsgemeinden.</p>
					<p>
						<add type="marginal" place="right">
							<lb facs="#facs_140_r33"/>Änderung<lb facs="#facs_140_r34"/>der Anhänge</add>
					</p>
					<p>
						<lb facs="#facs_140_r35"/>§ 18. Für die Änderung des Anhanges 1 (Streckennetz) ist<lb facs="#facs_140_r36"/>die Zustimmung aller Vorortsgemeinden und des <orgName n="ff7c9811-0c04-441c-bc98-110c04541c26">Regierungs<lb break="no" facs="#facs_140_r37"/>rates</orgName> erforderlich.</p>
					<p>
						<lb facs="#facs_140_r38"/>Für die Änderung des Anhanges 2 (Verteilschlüssel) ist die<lb facs="#facs_140_r39"/>Zustimmung aller Vorortsgemeinden erforderlich.</p>
					<p>
						<lb facs="#facs_140_r40"/>Für die Änderung des Anhanges 3 (beitragsberechtigte<lb facs="#facs_140_r41"/>Anschlussstrecken des VBZ-Netzes) ist nur die Zustimmung<lb facs="#facs_140_r42"/>der Vertreter des <orgName n="ef78e454-41b7-4993-b8e4-5441b7b993b8">Kantons</orgName> und der Stadt <placeName n="ab33f713-35bb-4ee6-b3f7-1335bb7ee695">Zürich</placeName> in der Be<lb break="no" facs="#facs_140_r43"/>triebskommission erforderlich.</p>
					<p>
						<add type="marginal" place="right">
							<lb facs="#facs_140_r44"/>Inkrafttreten</add>
					</p>
					<p>
						<lb facs="#facs_140_r45"/>§ 19. Dieser Gründungsvertrag tritt nach der Annahme<lb facs="#facs_140_r46"/>durch die beteiligten Gemeinden und nach dem unbenützten<lb facs="#facs_140_r47"/>Ablauf der Referendumsfrist zum Genehmigungsbeschluss <orgName n="b9361870-9590-4dad-b618-709590fdad01">des<lb facs="#facs_140_r48"/>Kantonsrates</orgName> auf den ersten Tag des folgenden Monats in<lb facs="#facs_140_r49"/>Kraft.</p>
				</div>
			</div>
			<pb n="140" facs="#facs_141"/>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_141_r1"/>Anhang 1<lb facs="#facs_141_r2"/>(zu § 2 Abs. 2)</head>
				<p>
					<lb facs="#facs_141_r3"/>Die Verkehrsbetriebe der Region Unterer Zimmerberg<lb facs="#facs_141_r4"/>unterhalten das folgende Streckennetz:</p>
				<table>
					<row>
						<cell>
							<lb facs="#facs_141_r5"/>Linie 61:</cell>
						<cell>
							<lb facs="#facs_141_r6"/><placeName n="f9a603d9-e15d-4fdf-a603-d9e15d0fdf21">Zürich</placeName> (Bürkliplatz)-Seestrasse-Alte Landstrasse<lb facs="#facs_141_r7"/>-Kirche Kilchberg</cell>
					</row>
					<row>
						<cell>
							<lb facs="#facs_141_r8"/>Linie 65:</cell>
						<cell>
							<lb facs="#facs_141_r9"/><placeName n="976e187d-7536-4f00-ae18-7d7536af009e">Zürich</placeName> (Bürkliplatz)-Seestrasse-Nidelbad <placeName n="bf2f1d5b-0683-4338-af1d-5b0683833875">Rüschli<lb break="no" facs="#facs_141_r10"/>kon</placeName>kon</cell>
					</row>
					<row>
						<cell>
							<lb facs="#facs_141_r11"/>Linie 84:</cell>
						<cell>
							<lb facs="#facs_141_r12"/>Zürich-Wollishofen (Endstation der Tramlinie 7)-<lb facs="#facs_141_r13"/>Albisstrasse/Zürichstrasse-Krone Adliswil-Kilch<lb break="no" facs="#facs_141_r14"/>bergstrasse-Grütstrasse-Spital Sanitas Kilchberg</cell>
					</row>
				</table>
			</div>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_141_r15"/>Anhang 2<lb facs="#facs_141_r16"/>(zu § 4 Abs. 1)</head>
				<p>
					<lb facs="#facs_141_r17"/>Das Betriebsdefizit der Buslinien in der Region <placeName n="759d49f1-229a-49f7-9d49-f1229af9f7ec">Unterer<lb facs="#facs_141_r18"/>Zimmerberg</placeName> wird nach folgendem Schlüssel verteilt:</p>
				<p>
					<lb facs="#facs_141_r19"/>98 % Aufwandfaktor</p>
				<p>
					<lb facs="#facs_141_r20"/>1 % Nettosteuerkraft pro Einwohner</p>
				<p>
					<lb facs="#facs_141_r21"/>1 % Einwohnerzahl.</p>
				<p>
					<lb facs="#facs_141_r22"/>Übergangsbestimmung:</p>
				<p>
					<lb facs="#facs_141_r23"/>Während drei Jahren ab Gründung wird der Beitrag der<lb facs="#facs_141_r24"/>Stiftung <orgName n="069d1f86-da2a-4c2c-9d1f-86da2a0c2c23">Krankenhaus Sanitas</orgName> für die Linie 84 nicht der für die<lb facs="#facs_141_r25"/>Verteilung massgebenden Betriebsrechnung gutgeschrieben,<lb facs="#facs_141_r26"/>sondern unter der Stadt <placeName n="b156279c-9fe9-40bd-9627-9c9fe960bd7c">Adliswil</placeName> und der Gemeinde Kilchberg<lb facs="#facs_141_r27"/>verteilt.</p>
			</div>
			<div type="subtitle" n="1">
				<head type="subtitle" n="1">
					<lb facs="#facs_141_r28"/>Anhang 3<lb facs="#facs_141_r29"/>(zu § 4 Abs. 2)</head>
				<p>
					<lb facs="#facs_141_r30"/>Für die Berechnung des auf die <orgName n="af308bd7-3b95-47ef-b08b-d73b9557ef4f">VBZ</orgName> entfallenden Be<lb break="no" facs="#facs_141_r31"/>triebsdefizites wird folgender Linienabschnitt berücksichtigt:</p>
				<p>
					<lb facs="#facs_141_r32"/>7 <placeName n="bdb92547-a2aa-4543-b925-47a2aaa543c7">Wollishofen-Paradeplatz</placeName></p>
				<p>
					<lb facs="#facs_141_r33"/>Den Einnahmen auf diesem Linienabschnitt wird die Tax<lb break="no" facs="#facs_141_r34"/>ordnung der Stadt <placeName n="67220232-a9d0-4af1-a202-32a9d0aaf1bd">Zürich</placeName> zugrunde gelegt.</p>
			</div>
		</body></text></TEI>
"""

In [94]:
s = BeautifulSoup(i, 'xml')

In [95]:
print(s.title)

<title type="main">Amtsblatt des Kantons Zürich</title>


In [96]:
print(s.title.getText())

Amtsblatt des Kantons Zürich


In [33]:
print(s.find_all('placeName'))

[<placeName n="a5ddb64d-059e-48f7-9db6-4d059ee8f791">Zürich</placeName>, <placeName n="cd0b3bca-3ea6-416b-8b3b-ca3ea6c16bfe">Adliswil</placeName>, <placeName n="97d155da-4b5f-4cdc-9155-da4b5ffcdc15">Kilchberg</placeName>, <placeName n="fdc817b1-c431-4aad-8817-b1c4313aade3">Rüschlikon</placeName>, <placeName n="c3bb6527-1ba5-44a6-bb65-271ba574a69a">Zürich</placeName>, <placeName n="16809273-5651-426c-8092-735651c26cb9">Kilchberg.</placeName>, <placeName n="4827755e-a4e4-4cbb-a775-5ea4e48cbb61">Unterer Zimmerberg</placeName>, <placeName n="e8b74d0c-bf10-4ddc-b74d-0cbf10bddc8c" type="deriv">Zürcher</placeName>, <placeName n="f968ca3e-0feb-4ec8-a8ca-3e0feb1ec83a">Zürich</placeName>, <placeName n="0ef30930-8296-42d8-b309-308296d2d872">Unterer Zimmerberg</placeName>, <placeName n="6c0e08a8-ca62-4c88-8e08-a8ca621c8804">Zürich.</placeName>, <placeName n="e387820b-5501-4ba4-8782-0b5501dba42d">Unterer Zimmer<lb break="no" facs="#facs_136_r21"/>berg</placeName>, <placeName n="da17a764-b353-4bf0-9

In [22]:
input = '<p><lb facs="#facs_136_r6"/>Die Stadt Zürich beteiligt sich entsprechend dem Betriebs<lb break="no" facs="#facs_136_r7"/>defizit, welches auf die im Anhang 3 zum Gründungsvertrag<lb facs="#facs_136_r8"/>aufgeführten Bus -, Trolleybus- und Tramlinien entfällt, wel<lb break="no" facs="#facs_136_r9"/>che die Region Unterer Zimmerberg mit dem Stadtzentrum<lb facs="#facs_136_r10"/>verbinden. Dabei werden nur die Kurse berücksichtigt, welche<lb facs="#facs_136_r11"/>den Anschluss an Kurse aus der Region Unterer Zimmerberg<lb facs="#facs_136_r12"/>herstellen.</p>'

print(input)

<p><lb facs="#facs_136_r6"/>Die Stadt Zürich beteiligt sich entsprechend dem Betriebs<lb break="no" facs="#facs_136_r7"/>defizit, welches auf die im Anhang 3 zum Gründungsvertrag<lb facs="#facs_136_r8"/>aufgeführten Bus -, Trolleybus- und Tramlinien entfällt, wel<lb break="no" facs="#facs_136_r9"/>che die Region Unterer Zimmerberg mit dem Stadtzentrum<lb facs="#facs_136_r10"/>verbinden. Dabei werden nur die Kurse berücksichtigt, welche<lb facs="#facs_136_r11"/>den Anschluss an Kurse aus der Region Unterer Zimmerberg<lb facs="#facs_136_r12"/>herstellen.</p>


In [200]:
original = """
<xml>
<lb facs="#facs_135_r17"/>gründen hiermit unter Vorbehalt der Änderung des Kreises<lb facs="#facs_135_r18"/>der Beteiligten die Unternehmung «Verkehrsbetriebe der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg (VRUZ)» als regionalen Ver<lb break="no" facs="#facs_135_r20"/>kehrsbetrieb im Sinne der §§ 4-13 des Gesetzes über den<lb facs="#facs_135_r21"/>regionalen öffentlichen Verkehr.</p>
</xml>
"""

input = """
<xml>
<lb facs="#facs_135_r17"/>gründen hiermittttttttttttt hgvhjfj hgfjfgh unter Vorbehalt der Änderung des Kreises<lb facs="#facs_135_r18"/>der Beteiligten die Unternehmung «<orgName n="f9afe839-23e0-4b0e-afe8-3923e03b0e18">knl der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg</orgName> (<orgName n="473cba5f-8e66-40b9-bcba-5f8e66b0b9f5">VRUZ</orgName>)» als regionalen Ver<lb break="no" facs="#facs_135_r20"/>kehrsbetrieb im Sinne der §§ 4-13 des Gesetzes über den<lb facs="#facs_135_r21"/>regionalen öffentlichen Verkehr.</p>
</xml>
"""

In [182]:
from bs4 import BeautifulSoup
i = BeautifulSoup(input, 'xml')

print(i)
print(i.next_element)

<?xml version="1.0" encoding="utf-8"?>
<div n="1" type="subtitle">
<head n="1" type="subtitle">
<lb facs="#facs_141_r28"/>Anhang 3<lb facs="#facs_141_r29"/>(zu § 4 Abs. 2)</head>
<p>
<lb facs="#facs_141_r30"/>Für die Berechnung des auf die <orgName n="af308bd7-3b95-47ef-b08b-d73b9557ef4f">VBZ</orgName> entfallenden Be<lb break="no" facs="#facs_141_r31"/>triebsdefizites wird folgender Linienabschnitt berücksichtigt:</p>
<p>
<lb facs="#facs_141_r32"/>7 <placeName n="bdb92547-a2aa-4543-b925-47a2aaa543c7">Wollishofen-Paradeplatz</placeName></p>
<p>
<lb facs="#facs_141_r33"/>Den Einnahmen auf diesem Linienabschnitt wird die Tax<lb break="no" facs="#facs_141_r34"/>ordnung der Stadt <placeName n="67220232-a9d0-4af1-a202-32a9d0aaf1bd">Zürich</placeName> zugrunde gelegt.</p>
<p>
<lb facs="#facs_141_r30"/>Für die Berechnung des auf die <orgName n="af308bd7-3b95-47ef-b08b-d83b9557ef4f">Verkehrsbetriebe <placeName n="bdb92544-a2aa-4543-b925-47a2aaa543c7">Zürich</placeName></orgName> entfallenden B

In [188]:
from bs4 import BeautifulSoup

i = BeautifulSoup(input, 'xml')

placeNames = i.find_all('placeName')

print(placeNames)


[<placeName n="bdb92547-a2aa-4543-b925-47a2aaa543c7">Wollishofen-Paradeplatz</placeName>, <placeName n="67220232-a9d0-4af1-a202-32a9d0aaf1bd">Zürich</placeName>, <placeName n="bdb92544-a2aa-4543-b925-47a2aaa543c7">Zürich</placeName>]


In [219]:
## ChatGPT


from bs4 import BeautifulSoup

def merge_entities(original_xml, edited_xml):
    # Parsen des Original- und bearbeiteten XMLs
    original_soup = BeautifulSoup(original_xml, 'xml')
    edited_soup = BeautifulSoup(edited_xml, 'xml')

    # Liste aller Entitäten im bearbeiteten XML finden (z.B. Orte, Personen, Organisationen)
    entities = edited_soup.find_all(['placeName', 'persName', 'orgName'])

    # Iteriere über jede Entität im bearbeiteten XML
    for entity in entities:
        # Extrahiere den Text der markierten Entität
        entity_text = entity.get_text()

        # Suche nach der ersten Instanz des Textes im Original-XML
        for text_element in original_soup.find_all(string=True):
            if entity_text in text_element:
                # Extrahiere den ursprünglichen Text und teile ihn
                text_before, text_after = text_element.split(entity_text, 1)

                # Erstelle das neue Tag für die Entität aus dem bearbeiteten XML
                new_entity_tag = original_soup.new_tag(entity.name, **entity.attrs)
                new_entity_tag.string = entity_text

                # Baue die neue Struktur: Text vor der Entität, die Entität selbst, und der Text danach
                new_text_before = original_soup.new_string(text_before)
                new_text_after = original_soup.new_string(text_after)

                # Ersetze den ursprünglichen Text durch die neue Struktur
                text_element.replace_with(new_text_before)
                new_text_before.insert_after(new_entity_tag)
                new_entity_tag.insert_after(new_text_after)

                # Beende die Schleife, um nur die erste Instanz zu ersetzen
                break

    return str(original_soup)

# Beispiel-XMLs
original_xml = '''<text>Ich reise nach Zürich. Peter Müller arbeitet in der Firma ABC Zürich.</text>'''
edited_xml = '''<text>Ich reise nach <placeName n="1">Zürich</placeName>. <persName n="2">Peter Müller</persName> arbeitet in der Firma <orgName n="3">ABC <placeName n="4">Zürich</placeName></orgName>.</text>'''

# Ergebnis
result = merge_entities(original_xml, edited_xml)
print(result)


<?xml version="1.0" encoding="utf-8"?>
<text>Ich reise nach <placeName n="1"><placeName n="4">Zürich</placeName></placeName>. <persName n="2">Peter Müller</persName> arbeitet in der Firma <orgName n="3">ABC Zürich</orgName>.</text>


In [198]:
original_xml = """
    <div type="subtitle" n="1">
        <head type="subtitle" n="1">
            <lb facs="#facs_141_r28"/>Anhang 3<lb facs="#facs_141_r29"/>(zu § 4 Abs. 2)</head>
        <p>
            <lb facs="#facs_141_r30"/>Für die Berechnung des auf die VBZ entfallenden Be<lb break="no" facs="#facs_141_r31"/>triebsdefizites wird folgender Linienabschnitt berücksichtigt:</p>
        <p>
            <lb facs="#facs_141_r32"/>7 Wollishofen-Paradeplatz</p>
        <p>
            <lb facs="#facs_141_r33"/>Den Einnahmen auf diesem Linienabschnitt wird die Tax<lb break="no" facs="#facs_141_r34"/>ordnung der Stadt Zürich zugrunde gelegt.</p>
        <p>
            <lb facs="#facs_141_r30"/>Für die Berechnung des auf die Verkehrsbetriebe Zürich entfallenden Be<lb break="no" facs="#facs_141_r31"/>triebsdefizites wird folgender Linienabschnitt berücksichtigt:</p>
    </div>
"""

edited_xml = """
    <div type="subtitle" n="1">
        <head type="subtitle" n="1">
            <lb facs="#facs_141_r28"/>Anhang 3<lb facs="#facs_141_r29"/>(zu § 4 Abs. 2)</head>
        <p>
            <lb facs="#facs_141_r30"/>Für die Berechnung des auf die <orgName n="af308bd7-3b95-47ef-b08b-d73b9557ef4f">VBZ</orgName> entfallenden Be<lb break="no" facs="#facs_141_r31"/>triebsdefizites wird folgender Linienabschnitt berücksichtigt:</p>
        <p>
            <lb facs="#facs_141_r32"/>7 <placeName n="bdb92547-a2aa-4543-b925-47a2aaa543c7">Wollishofen-Paradeplatz</placeName></p>
        <p>
            <lb facs="#facs_141_r33"/>Den Einnahmen auf diesem Linienabschnitt wird die Tax<lb break="no" facs="#facs_141_r34"/>ordnung der Stadt <placeName n="67220232-a9d0-4af1-a202-32a9d0aaf1bd">Zürich</placeName> zugrunde gelegt.</p>
        <p>
            <lb facs="#facs_141_r30"/>Für die Berechnung des auf die <orgName n="af308bd7-3b95-47ef-b08b-d83b9557ef4f">Verkehrsbetriebe <placeName n="bdb92544-a2aa-4543-b925-47a2aaa543c7">Zürich</placeName></orgName> entfallenden Be<lb break="no" facs="#facs_141_r31"/>triebsdefizites wird folgender Linienabschnitt berücksichtigt:</p>
    </div>
"""

In [201]:
import re
from bs4 import BeautifulSoup
i = BeautifulSoup(input, 'xml')

idsReplaced = []

placeNames = i.find_all('placeName')
orgNames = i.find_all('orgName')
#print(orgNames)
#print(input)

#take only inner text of placeName and search with this text in original and replace placeName. save in output

output = original
for placeName in placeNames:
    #print(placeName.string)

    for org_tag in placeName.find_all('placeName'):
        org_tag.unwrap()

    for org_tag in placeName.find_all('orgName'):
        org_tag.unwrap()
    
    toReplace = ''.join(str(content) for content in placeName.contents)

    print(toReplace)
    #replacement = str(placeName.append(' --Done--'))
    replacement = str(placeName)
    print(replacement)

    pattern = rf'(?<!<placeName n="[^"]+">){re.escape(toReplace)}'

    
    # Text ersetzen
    output = re.sub(pattern, replacement, output)

    #output = output.replace(toReplace, replacement, 1)
    #print(str(placeName))

for orgName in orgNames:
    
    #print(type(orgName.encode_contents()))
    orgName["done"]= "true"
    print(orgName)

    # Alle orgName-Tags finden und deren Inhalt an die Stelle des orgName-Tags setzen
    for org_tag in orgName.find_all('placeName'):
        org_tag.unwrap()

    for org_tag in orgName.find_all('orgName'):
        org_tag.unwrap()
    
    # Ausgabe des Strings ohne orgName-Tags
    print(str(orgName))
    toReplace = ''.join(str(content) for content in orgName.contents)
    

    #replace all other tei tags
    #og = BeautifulSoup(orgName, 'xml')
    #for pl_tag in og.find_all('placeName'):
    #    pl_tag.unwrap()
    #
    print(toReplace)
    output = output.replace(toReplace, str(orgName), 1)
    #print(str(orgName))


print(original)
print('---------------------')
print(output)

<orgName done="true" n="f9afe839-23e0-4b0e-afe8-3923e03b0e18">knl der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg</orgName>
<orgName done="true" n="f9afe839-23e0-4b0e-afe8-3923e03b0e18">knl der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg</orgName>
knl der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg
<orgName done="true" n="473cba5f-8e66-40b9-bcba-5f8e66b0b9f5">VRUZ</orgName>
<orgName done="true" n="473cba5f-8e66-40b9-bcba-5f8e66b0b9f5">VRUZ</orgName>
VRUZ

<xml>
<lb facs="#facs_135_r17"/>gründen hiermit unter Vorbehalt der Änderung des Kreises<lb facs="#facs_135_r18"/>der Beteiligten die Unternehmung «Verkehrsbetriebe der<lb facs="#facs_135_r19"/>Region Unterer Zimmerberg (VRUZ)» als regionalen Ver<lb break="no" facs="#facs_135_r20"/>kehrsbetrieb im Sinne der §§ 4-13 des Gesetzes über den<lb facs="#facs_135_r21"/>regionalen öffentlichen Verkehr.</p>
</xml>

---------------------

<xml>
<lb facs="#facs_135_r17"/>gründen hiermit unter Vorbehalt der Änderung des Kreise

In [330]:
from bs4 import BeautifulSoup


##Hilfsfunktionen
def filter_nested_entities(entities):
    # Eine Liste zur Speicherung der nicht verschachtelten Entitäten
    non_nested_entities = []

    # Hilfsfunktion zum Überprüfen, ob ein Tag innerhalb eines anderen Tags vorkommt
    def is_nested(tag):
        parent = tag.find_parent()
        while parent:
            if parent.name in {'placeName', 'persName', 'orgName'} and parent != tag:
                return True
            parent = parent.find_parent()
        return False

    # Iteriere über alle Entitäten
    for entity in entities:
        # Überprüfen, ob das Tag verschachtelt ist
        if not is_nested(entity):
            non_nested_entities.append(entity)

    return non_nested_entities

def prepare_search_text(entity):
    #print(entity.name)

    # Ermitteln des Tag-Namens
    entity_name = entity.name
    
    # Ermitteln der ID (attribut 'n')
    entity_id = entity.get('n', 'No ID found')  # 'n' ist der Name des Attributs

    print(entity)
    
    def remove_entities_within(tag):
        # Suche nach den Entitäten innerhalb des Tags
        entities = tag.find_all(['placeName', 'persName', 'orgName'])
        for ent in entities:
            # Ersetze das Entitätstag durch seinen Inhalt
            ent.replace_with(ent.contents)

    # Entferne alle Entitäten innerhalb der übergebenen Entity
    remove_entities_within(entity)
    
    return str(entity)
    
    
    
    toReplace = ''.join(str(content) for content in entity.contents)
    


def merge_entities(original_xml, edited_xml):
    # Parsen des Original- und bearbeiteten XMLs
    original_soup = BeautifulSoup(original_xml, 'xml')
    edited_soup = BeautifulSoup(edited_xml, 'xml')

    # Liste aller Entitäten im bearbeiteten XML finden (z.B. Orte, Personen, Organisationen)
    entities = edited_soup.find_all(['placeName', 'persName', 'orgName'])

    #Entferne in der Liste alle Entitäten, die bereits verschachtelt vorkommen
    non_nested_entities = filter_nested_entities(entities)
    
    # Iteriere über jede Entität im bearbeiteten XML
    for entity in non_nested_entities:
        
        searchText = prepare_search_text(entity)
        
        

    return str(original_soup)

# Beispiel-XMLs
original_xml = '''<text>Ich reise nach <i>Zürich</i>. Peter Müller arbeitet in der Firma ABC Zürich.</text>'''
edited_xml = '''<text>Ich reise nach <placeName n="1"><i>Zürich</i></placeName> im Kanton <placeName n="2"><i>Zürich</i></placeName>. <persName n="2">Peter Müller</persName> arbeitet in der Firma <orgName n="3">ABC <persName n="4">Dr. <b>Gerhard</b></persName> <placeName n="4">Zürich</placeName></orgName>.</text>'''

# Ergebnis
result = merge_entities(original_xml, edited_xml)
#print(result)


<placeName n="1"><i>Zürich</i></placeName>
<placeName n="2"><i>Zürich</i></placeName>
<persName n="2">Peter Müller</persName>
<orgName n="3">ABC <persName n="4">Dr. <b>Gerhard</b></persName> <placeName n="4">Zürich</placeName></orgName>


AttributeError: 'list' object has no attribute 'parent'

In [293]:
from bs4 import BeautifulSoup

data = """
<html>
 <head>
    <title>
     insert title here
    </title>
 </head>
</html>
"""

soup = BeautifulSoup(data, "html.parser")
print(soup.find())

html
